In [1]:
import requests

# Eurostat API endpoint for 'nrg_ind_ren' dataset
DATA_URL = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_ind_ren?format=SDMX-CSV&compressed=false"
OUTPUT_FILE = "../data/raw/nrg_ind_ren_linear.csv"

print(f"Fetching data from Eurostat...")

try:
    response = requests.get(DATA_URL)
    response.raise_for_status()
    
    with open(OUTPUT_FILE, 'wb') as f:
        f.write(response.content)
    
    print(f"Data saved to {OUTPUT_FILE}")

except requests.RequestException as e:
    print(f"Download failed: {e}")

Fetching data from Eurostat...
Data saved to ../data/raw/nrg_ind_ren_linear.csv


In [2]:
import pandas as pd

# Load raw Eurostat data
df = pd.read_csv('../data/raw/nrg_ind_ren_linear.csv')

# Configuration: Map technical codes to readable labels (Italian for viz)
SECTOR_MAP = {
    'REN': 'Totale (Overall)',
    'REN_E': 'Elettricità',
    'REN_T': 'Trasporti',
    'REN_HC': 'Riscaldamento & Raffreddamento'
}

# Select relevant columns and drop missing values
cols = ['nrg_bal', 'geo', 'TIME_PERIOD', 'OBS_VALUE']
df = df[cols].dropna(subset=['OBS_VALUE'])

# Filter rows based on our sector interest and map labels
df = df[df['nrg_bal'].isin(SECTOR_MAP.keys())].copy()
df['Settore'] = df['nrg_bal'].map(SECTOR_MAP)

print(f"Dataset cleaned. Shape: {df.shape}")
print(df.head())

Dataset cleaned. Shape: (773, 5)
  nrg_bal geo  TIME_PERIOD  OBS_VALUE           Settore
0     REN  AL         2004     29.620  Totale (Overall)
1     REN  AL         2005     31.367  Totale (Overall)
2     REN  AL         2006     32.070  Totale (Overall)
3     REN  AL         2007     32.657  Totale (Overall)
4     REN  AL         2008     32.448  Totale (Overall)


In [3]:
import pandas as pd
import os
import sys

# Source: https://w3.unece.org/SDG/en/Indicator?id=23
# Note: Download csv manually to this folder
INPUT_FILE = '../data/raw/sdg_out_000016340_i23_en.csv'
OUTPUT_FILE = '../data/output/dati_unece_iso_clean.csv'

COUNTRY_TO_ISO = {
    'Albania': 'AL', 'Andorra': 'AD', 'Armenia': 'AM', 'Austria': 'AT', 
    'Azerbaijan': 'AZ', 'Belarus': 'BY', 'Belgium': 'BE', 
    'Bosnia and Herzegovina': 'BA', 'Bulgaria': 'BG', 'Croatia': 'HR', 
    'Cyprus': 'CY', 'Czechia': 'CZ', 'Denmark': 'DK', 'Estonia': 'EE', 
    'Finland': 'FI', 'France': 'FR', 'Georgia': 'GE', 'Germany': 'DE', 
    'Greece': 'GR', 'Hungary': 'HU', 'Iceland': 'IS', 'Ireland': 'IE', 
    'Israel': 'IL', 'Italy': 'IT', 'Kazakhstan': 'KZ', 'Kyrgyzstan': 'KG', 
    'Latvia': 'LV', 'Liechtenstein': 'LI', 'Lithuania': 'LT', 
    'Luxembourg': 'LU', 'Malta': 'MT', 'Montenegro': 'ME', 
    'Netherlands': 'NL', 'North Macedonia': 'MK', 'Norway': 'NO', 
    'Poland': 'PL', 'Portugal': 'PT', 'Republic of Moldova': 'MD', 'Moldova': 'MD',
    'Romania': 'RO', 'Russian Federation': 'RU', 'Serbia': 'RS', 
    'Slovakia': 'SK', 'Slovenia': 'SI', 'Spain': 'ES', 'Sweden': 'SE', 
    'Switzerland': 'CH', 'Tajikistan': 'TJ', 'Türkiye': 'TR', 'Turkey': 'TR',
    'Turkmenistan': 'TM', 'Ukraine': 'UA', 'United Kingdom': 'GB', 
    'Uzbekistan': 'UZ', 'United States': 'US', 'Canada': 'CA'
}

BLACKLIST = ['US', 'CA', 'UZ', 'TM', 'TJ', 'KG', 'KZ', 'IL', 'AM', 'AZ', 'GE']

if not os.path.exists(INPUT_FILE):
    print(f"Error: File '{INPUT_FILE}' not found.")
    print("Please download it from https://w3.unece.org/SDG/en/Indicator?id=23")
    sys.exit(1)

print(f"Reading {INPUT_FILE}...")

# 1. Load Data
try:
    df = pd.read_csv(INPUT_FILE, header=1, encoding='latin1')
except UnicodeDecodeError:
    df = pd.read_csv(INPUT_FILE, header=1, encoding='cp1252')

# 2. Pre-processing
df.columns = df.columns.str.strip()
df['Period'] = pd.to_numeric(df['Period'], errors='coerce')
df['Value'] = pd.to_numeric(df['Value'], errors='coerce')

# 3. Filter: Most recent year per country
df = df.sort_values(['Country_E', 'Period'], ascending=[True, False])
df = df.drop_duplicates(subset=['Country_E'], keep='first').copy()

# 4. Map ISO codes and filter blacklist
df['iso_code'] = df['Country_E'].map(COUNTRY_TO_ISO)
df = df[df['iso_code'].notna()]
df = df[~df['iso_code'].isin(BLACKLIST)]

# 5. Export
df[['iso_code', 'Value']].to_csv(OUTPUT_FILE, index=False)

print(f"Done. Exported to {OUTPUT_FILE}")

Reading ../data/raw/sdg_out_000016340_i23_en.csv...
Done. Exported to ../data/output/dati_unece_iso_clean.csv


In [4]:
import pandas as pd

# Source: Our World in Data (OWID)
DATA_URL = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
OUTPUT_FILE = "../data/output/italy_energy_mix_1990-2023.csv"
START_YEAR = 1990

# Map OWID columns to display labels
COLUMN_MAPPING = {
    'year': 'Year',
    'coal_electricity': 'Coal',
    'gas_electricity': 'Gas',
    'oil_electricity': 'Oil',
    'hydro_electricity': 'Hydro',
    'solar_electricity': 'Solar',
    'wind_electricity': 'Wind',
    'biofuel_electricity': 'Bioenergy',
    'nuclear_electricity': 'Nuclear',
    'other_renewable_electricity': 'Other Renewables'
}

def generate_energy_mix_csv():
    print("Downloading dataset from Our World in Data...")
    
    try:
        df = pd.read_csv(DATA_URL)
        
        # Filter: Country and Time Range
        df = df[(df['country'] == 'Italy') & (df['year'] >= START_YEAR)].copy()
        
        # Select relevant columns that exist in the dataset
        available_cols = [c for c in COLUMN_MAPPING.keys() if c in df.columns]
        df = df[available_cols].rename(columns=COLUMN_MAPPING)
        
        # Data Cleaning: Fill NaN with 0 for cleaner stack charts
        df = df.fillna(0)
        
        # Sort chronologically
        df = df.sort_values(by='Year')
        
        # Export
        df.to_csv(OUTPUT_FILE, index=False)
        
        print(f"Done. File saved to: {OUTPUT_FILE}")
        print(f"Range: {df['Year'].min()} - {df['Year'].max()}")
        print(f"Shape: {df.shape}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    generate_energy_mix_csv()

Done. File saved to: ../data/output/italy_energy_mix_1990-2023.csv
Range: 1990 - 2025
Shape: (36, 10)


In [5]:
import requests

# Eurostat full dataset endpoint (compressed TSV format)
URL = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_ind_ren?format=TSV&compressed=true"
OUTPUT_FILE = "../data/raw/nrg_ind_ren_FULL.gz"

print(f"Starting download from Eurostat...")

try:
    # Set timeout to handle potential server lags
    response = requests.get(URL, timeout=120)
    response.raise_for_status()
    
    with open(OUTPUT_FILE, 'wb') as f:
        f.write(response.content)
        
    print(f"Download successful. File saved: {OUTPUT_FILE}")

except requests.RequestException as e:
    print(f"Error during download: {e}")

Starting download from Eurostat...
Download successful. File saved: ../data/raw/nrg_ind_ren_FULL.gz


In [6]:
import pandas as pd
import gzip
import os
import sys

# Configuration
INPUT_FILES = ['../data/raw/nrg_ind_ren_FULL.gz', '../data/raw/nrg_ind_ren.gz']
OUTPUT_FILE = '../data/output/eurostat_sector_gap_flourish.csv'
TARGET_GEO = 'EU27_2020'  # Fallback to similar code if not found

# Eurostat codes to readable English labels
SECTOR_MAP = {
    'REN_ELC': 'Electricity',
    'REN_TRA': 'Transport', 
    'REN_HEAT_CL': 'Heating & Cooling'
}

def get_input_file():
    """Finds the first available input file from the list."""
    for f in INPUT_FILES:
        if os.path.exists(f):
            return f
    return None

def process_eurostat_data():
    input_path = get_input_file()
    
    if not input_path:
        print(f"Error: Input file not found. Checked: {INPUT_FILES}")
        sys.exit(1)

    print(f"Processing {input_path}...")

    try:
        # 1. Load compressed TSV
        # Eurostat TSVs often have a complex first column containing multiple keys
        with gzip.open(input_path, 'rt') as f:
            df = pd.read_csv(f, sep='\t', dtype=str)

        # 2. Parse the composite index column (freq,nrg_bal,unit,geo\TIME_PERIOD)
        # Example format: "A,REN,kTOE,AT\2022"
        first_col = df.columns[0]
        meta_data = df[first_col].str.split(',', expand=True)
        
        # Assign proper columns
        df['nrg_bal'] = meta_data[1]
        df['geo'] = meta_data[3].str.split(r'\\').str[0].str.strip()
        
        # Drop the complex index after extraction
        df.drop(columns=[first_col], inplace=True)

        # 3. Filter by Geography (EU27)
        if TARGET_GEO not in df['geo'].unique():
            # Fallback: look for any EU27 code if the specific version changed
            alternatives = [g for g in df['geo'].unique() if 'EU27' in str(g)]
            geo_filter = alternatives[0] if alternatives else TARGET_GEO
            print(f"Notice: '{TARGET_GEO}' not found. Using '{geo_filter}' instead.")
        else:
            geo_filter = TARGET_GEO
            
        df = df[df['geo'] == geo_filter].copy()

        # 4. Filter by Sector
        df = df[df['nrg_bal'].isin(SECTOR_MAP.keys())]

        if df.empty:
            print("Warning: No matching sectors found. Check if dataset is aggregated (REN only).")
            return

        # 5. Reshape (Melt) - Convert Years from columns to rows
        # Identify year columns (everything that is not metadata)
        id_vars = ['nrg_bal', 'geo']
        value_vars = [c for c in df.columns if c not in id_vars and 'freq' not in c and 'unit' not in c]
        
        df_melted = df.melt(
            id_vars=['nrg_bal'],
            value_vars=value_vars,
            var_name='Year',
            value_name='Value'
        )

        # 6. Clean Values
        # Remove flags (e.g. "12.5 p" -> "12.5") and handle non-numeric
        df_melted['Value'] = df_melted['Value'].astype(str).str.split().str[0]
        df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')
        
        # Clean Year and ensure integer type
        df_melted['Year'] = df_melted['Year'].str.strip()
        df_melted = df_melted[df_melted['Year'].str.match(r'^\d{4}$', na=False)]
        df_melted['Year'] = df_melted['Year'].astype(int)

        # Remove rows with no data
        df_melted = df_melted.dropna(subset=['Value'])

        # 7. Pivot for Final Output (Year x Sectors)
        df_pivot = df_melted.pivot_table(
            index='Year',
            columns='nrg_bal',
            values='Value',
            aggfunc='mean'
        ).reset_index()

        # Rename columns to English
        df_pivot.rename(columns=SECTOR_MAP, inplace=True)
        df_pivot = df_pivot.sort_values('Year')

        # 8. Export
        df_pivot.to_csv(OUTPUT_FILE, index=False)
        
        print(f"Success. Data exported to {OUTPUT_FILE}")
        print(f"Years covered: {df_pivot['Year'].min()} - {df_pivot['Year'].max()}")
        print(df_pivot.head())

    except Exception as e:
        print(f"Critical error during processing: {e}")
        # In production, we might log the traceback here
        sys.exit(1)

if __name__ == "__main__":
    process_eurostat_data()

Processing ../data/raw/nrg_ind_ren_FULL.gz...
Success. Data exported to ../data/output/eurostat_sector_gap_flourish.csv
Years covered: 2004 - 2024
nrg_bal  Year  Electricity  Heating & Cooling  Transport
0        2004       15.871             11.735      1.432
1        2005       16.402             12.437      1.819
2        2006       16.879             13.210      2.472
3        2007       17.647             14.819      2.887
4        2008       18.526             15.325      4.133


In [7]:
import pandas as pd
import gzip
import os
import sys

# Configuration
INPUT_FILES = ['../data/raw/nrg_ind_ren_FULL.gz', '../data/raw/nrg_ind_ren.gz']
OUTPUT_FILE = '../data/output/eurostat_regional_trends.csv'

# Regional Grouping (Excluded UK and CH as requested)
REGIONS = {
    'Nordics': ['SE', 'FI', 'DK', 'NO', 'IS'], 
    'Western Europe': ['DE', 'FR', 'BE', 'NL', 'LU', 'AT', 'IE'],
    'Southern Europe': ['IT', 'ES', 'PT', 'EL', 'MT', 'CY', 'HR', 'SI'],
    'Eastern Europe': ['PL', 'CZ', 'HU', 'SK', 'RO', 'BG', 'EE', 'LV', 'LT']
}

def get_input_file():
    for f in INPUT_FILES:
        if os.path.exists(f):
            return f
    return None

def process_regional_trends():
    input_path = get_input_file()
    
    if not input_path:
        print(f"Error: Input file not found. Checked: {INPUT_FILES}")
        sys.exit(1)

    print(f"Processing regional trends from {input_path}...")

    try:
        # 1. Load Data
        with gzip.open(input_path, 'rt') as f:
            df = pd.read_csv(f, sep='\t', dtype=str)

        # 2. Parse Metadata (Split first column)
        first_col = df.columns[0]
        meta = df[first_col].str.split(',', expand=True)
        
        df['nrg_bal'] = meta[1]
        df['geo'] = meta[3].str.split(r'\\').str[0].str.strip()
        df.drop(columns=[first_col], inplace=True)

        # 3. Create Country -> Region Map
        # Flatten the dictionary for easy mapping
        country_to_region = {code: region for region, codes in REGIONS.items() for code in codes}
        
        # 4. Filter Data
        # Keep only 'REN' (Overall Share) and countries in our regions
        mask = (df['nrg_bal'] == 'REN') & (df['geo'].isin(country_to_region.keys()))
        df = df[mask].copy()
        
        # Assign Region column
        df['Region'] = df['geo'].map(country_to_region)

        # 5. Melt (Wide to Long)
        # Identify year columns (digits only)
        id_vars = ['geo', 'Region', 'nrg_bal']
        value_vars = [c for c in df.columns if c not in id_vars and c.strip().isdigit()]
        
        df_melted = df.melt(
            id_vars=id_vars,
            value_vars=value_vars,
            var_name='Year',
            value_name='Value'
        )

        # 6. Clean Numeric Values
        # Handle Eurostat flags (e.g. "15.4 e") -> take first part, convert to float
        df_melted['Value'] = df_melted['Value'].astype(str).str.split().str[0]
        df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')
        
        # Clean Year
        df_melted['Year'] = df_melted['Year'].astype(int)
        
        # Drop missing values
        df_melted = df_melted.dropna(subset=['Value'])

        # 7. Aggregate (Mean per Region per Year)
        df_trends = df_melted.groupby(['Year', 'Region'])['Value'].mean().reset_index()

        # 8. Pivot (Year x Regions) for visualization tools
        df_final = df_trends.pivot(index='Year', columns='Region', values='Value').reset_index()
        df_final = df_final.sort_values('Year')

        # Export
        df_final.to_csv(OUTPUT_FILE, index=False)
        
        print(f"Success. Regional trends exported to {OUTPUT_FILE}")
        print(df_final.tail())

    except Exception as e:
        print(f"Error: {e}")
        sys.exit(1)

if __name__ == "__main__":
    process_regional_trends()

Processing regional trends from ../data/raw/nrg_ind_ren_FULL.gz...
Success. Regional trends exported to ../data/output/eurostat_regional_trends.csv
Region  Year  Eastern Europe  Nordics  Southern Europe  Western Europe
16      2020       23.485667  59.3654        22.615750       18.514571
17      2021       23.961889  60.2992        22.926000       17.732571
18      2022       24.734667  62.1824        23.070750       18.805857
19      2023       26.038889  63.4156        24.173375       21.055714
20      2024       27.233333  63.7390        24.531000       21.996571


In [8]:
import pandas as pd
import requests
import gzip
import os
import sys
import io

# Configuration
REN_FILES = ['../data/raw/nrg_ind_ren_FULL.gz', '../data/raw/nrg_ind_ren.gz']
DEP_FILE = '../data/raw/nrg_ind_id.tsv.gz'

# UPDATED URL: Using the SDMX API endpoint instead of the static file path (More robust)
DEP_URL = "https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/nrg_ind_id?format=TSV&compressed=true"

OUTPUT_FILE = '../data/output/war_impact_scatter.csv'
TARGET_YEAR = 2022

# Region Mapping
REGIONS = {
    'Nordics': ['SE', 'FI', 'DK', 'NO', 'IS'], 
    'Western Europe': ['DE', 'FR', 'BE', 'NL', 'LU', 'AT', 'IE', 'CH'],
    'Southern Europe': ['IT', 'ES', 'PT', 'EL', 'MT', 'CY', 'HR', 'SI'],
    'Eastern Europe': ['PL', 'CZ', 'HU', 'SK', 'RO', 'BG', 'EE', 'LV', 'LT']
}

def get_renewable_file():
    for f in REN_FILES:
        if os.path.exists(f):
            return f
    return None

def download_dependency_file():
    """Downloads the dependency dataset using the robust API endpoint."""
    print(f"Downloading data from Eurostat API to {DEP_FILE}...")
    try:
        # Increase timeout because API generation can take a few seconds
        r = requests.get(DEP_URL, stream=True, timeout=60)
        r.raise_for_status()
        
        with open(DEP_FILE, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        print("Download complete.")
    except Exception as e:
        print(f"Download failed: {e}")
        # Stop execution cleanly
        sys.exit(1)

def parse_eurostat_tsv(filepath, value_name):
    """
    Generic parser for Eurostat TSV.GZ files.
    """
    print(f"Parsing {filepath}...")
    try:
        with gzip.open(filepath, 'rt') as f:
            df = pd.read_csv(f, sep='\t', dtype=str)
    except EOFError:
        print(f"Error: The file {filepath} seems corrupted. Delete it and run again.")
        sys.exit(1)

    # 1. Parse Metadata
    first_col = df.columns[0]
    meta = df[first_col].str.split(',', expand=True)
    
    # Extract Geo (usually the last element in the comma-separated key)
    df['geo'] = meta.iloc[:, -1].str.split(r'\\').str[0].str.strip()
    
    # Keep nrg_bal if available
    if meta.shape[1] > 2:
        df['nrg_bal'] = meta[1]

    df.drop(columns=[first_col], inplace=True)

    # 2. Melt
    id_vars = ['geo', 'nrg_bal'] if 'nrg_bal' in df.columns else ['geo']
    value_vars = [c for c in df.columns if c.strip().isdigit()]
    
    df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='Year', value_name=value_name)

    # 3. Clean Numeric Data
    df_melted[value_name] = df_melted[value_name].astype(str).str.split().str[0]
    df_melted[value_name] = pd.to_numeric(df_melted[value_name], errors='coerce')
    
    # 4. Clean Year
    df_melted['Year'] = df_melted['Year'].astype(int)
    
    return df_melted.dropna(subset=[value_name])

def process_war_impact():
    # 1. Check/Load Renewables
    ren_file = get_renewable_file()
    if not ren_file:
        print("Error: Renewables file not found (nrg_ind_ren_FULL.gz). Run previous scripts first.")
        sys.exit(1)

    df_ren = parse_eurostat_tsv(ren_file, 'Renewables_Share')
    # Filter for 'REN' (Overall share)
    df_ren = df_ren[df_ren['nrg_bal'] == 'REN'].drop(columns=['nrg_bal'])

    # 2. Check/Download Dependency Data
    if not os.path.exists(DEP_FILE):
        download_dependency_file()
    
    df_dep = parse_eurostat_tsv(DEP_FILE, 'Import_Dependency')
    
    # 3. Filter Target Year
    print(f"Filtering data for year {TARGET_YEAR}...")
    
    # Check if 2022 exists, otherwise fallback to max year
    max_year_ren = df_ren['Year'].max()
    max_year_dep = df_dep['Year'].max()
    
    if TARGET_YEAR not in df_ren['Year'].unique():
        print(f"Warning: Year {TARGET_YEAR} not in Renewables. Using {max_year_ren}")
        df_ren_2022 = df_ren[df_ren['Year'] == max_year_ren]
    else:
        df_ren_2022 = df_ren[df_ren['Year'] == TARGET_YEAR]

    if TARGET_YEAR not in df_dep['Year'].unique():
        print(f"Warning: Year {TARGET_YEAR} not in Dependency. Using {max_year_dep}")
        df_dep_2022 = df_dep[df_dep['Year'] == max_year_dep]
    else:
        df_dep_2022 = df_dep[df_dep['Year'] == TARGET_YEAR]

    # 4. Merge
    print("Merging datasets...")
    df_final = pd.merge(df_ren_2022, df_dep_2022, on='geo', how='inner')

    # 5. Map Regions
    country_to_region = {code: region for region, codes in REGIONS.items() for code in codes}
    df_final['Region'] = df_final['geo'].map(country_to_region)
    
    # Remove unmapped countries
    df_final = df_final.dropna(subset=['Region'])

    # 6. Export
    final_cols = ['geo', 'Region', 'Renewables_Share', 'Import_Dependency']
    df_final[final_cols].to_csv(OUTPUT_FILE, index=False)

    print(f"Success. Data exported to {OUTPUT_FILE}")
    print(df_final.head())

if __name__ == "__main__":
    process_war_impact()

Parsing ../data/raw/nrg_ind_ren_FULL.gz...
Parsing ../data/raw/nrg_ind_id.tsv.gz...
Filtering data for year 2022...
Merging datasets...
Success. Data exported to ../data/output/war_impact_scatter.csv
   geo  Year_x  Renewables_Share          nrg_bal  Year_y  Import_Dependency  \
13  AT    2022            34.057  C0000X0350-0370    2022             99.905   
14  AT    2022            34.057            C0110    2022            125.010   
15  AT    2022            34.057            C0121    2022             98.084   
16  AT    2022            34.057            C0129    2022            101.695   
17  AT    2022            34.057            C0210    2022            100.000   

            Region  
13  Western Europe  
14  Western Europe  
15  Western Europe  
16  Western Europe  
17  Western Europe  


In [9]:
import pandas as pd
import gzip
import os
import sys

# Configuration
INPUT_FILES = ['../data/raw/nrg_ind_ren_FULL.gz', '../data/raw/nrg_ind_ren.gz']
OUTPUT_FILE = '../data/output/race_to_zero_growth.csv'
START_YEAR = 2012
END_YEAR = 2022

# Analysis Scope: A representative mix of EU economies
SELECTED_COUNTRIES = [
    'EU27_2020', 'SE', 'FI', 'DK', 'DE', 'FR', 
    'IT', 'ES', 'PL', 'RO', 'NL', 'BE'
]

# Label Mapping (for visualization tool tooltips/axis)
COUNTRY_LABELS = {
    'EU27_2020': 'EU Average', 
    'SE': 'Sweden', 'FI': 'Finland', 'DK': 'Denmark', 
    'DE': 'Germany', 'FR': 'France', 'IT': 'Italy', 
    'ES': 'Spain', 'PL': 'Poland', 'RO': 'Romania', 
    'NL': 'Netherlands', 'BE': 'Belgium'
}

def get_input_file():
    for f in INPUT_FILES:
        if os.path.exists(f):
            return f
    return None

def process_race_to_zero():
    input_path = get_input_file()
    
    if not input_path:
        print(f"Error: Input file not found. Checked: {INPUT_FILES}")
        sys.exit(1)

    print(f"Analyzing renewable growth ({START_YEAR}-{END_YEAR}) from {input_path}...")

    try:
        # 1. Load Data
        with gzip.open(input_path, 'rt') as f:
            df = pd.read_csv(f, sep='\t', dtype=str)

        # 2. Parse Metadata
        first_col = df.columns[0]
        meta = df[first_col].str.split(',', expand=True)
        
        df['nrg_bal'] = meta[1]
        df['geo'] = meta[3].str.split(r'\\').str[0].str.strip()
        df.drop(columns=[first_col], inplace=True)

        # 3. Filter Data
        # Keep only 'REN' (Total Renewables) and selected countries
        mask = (df['nrg_bal'] == 'REN') & (df['geo'].isin(SELECTED_COUNTRIES))
        df = df[mask].copy()

        # 4. Melt (Wide to Long)
        id_vars = ['nrg_bal', 'geo']
        value_vars = [c for c in df.columns if c not in id_vars and c.strip().isdigit()]
        
        df_melted = df.melt(id_vars=id_vars, value_vars=value_vars, var_name='Year', value_name='Value')

        # 5. Clean Values (Vectorized)
        # Remove flags, convert to numeric
        df_melted['Value'] = df_melted['Value'].astype(str).str.split().str[0]
        df_melted['Value'] = pd.to_numeric(df_melted['Value'], errors='coerce')
        
        # Clean Year
        df_melted['Year'] = df_melted['Year'].astype(int)

        # 6. Filter for Start and End Years
        df_final = df_melted[df_melted['Year'].isin([START_YEAR, END_YEAR])].copy()
        
        # 7. Pivot for Slope Chart format
        # Structure: Country | Value_Start | Value_End
        df_pivot = df_final.pivot(index='geo', columns='Year', values='Value').reset_index()
        
        # Rename columns dynamically based on config
        df_pivot.rename(columns={START_YEAR: f'Value_{START_YEAR}', END_YEAR: f'Value_{END_YEAR}'}, inplace=True)
        
        # 8. Enrich Data
        # Calculate absolute growth (percentage points)
        start_col = f'Value_{START_YEAR}'
        end_col = f'Value_{END_YEAR}'
        
        df_pivot['Growth_Points'] = df_pivot[end_col] - df_pivot[start_col]
        
        # Map nice names
        df_pivot['Country_Name'] = df_pivot['geo'].map(COUNTRY_LABELS).fillna(df_pivot['geo'])
        
        # Sort by growth
        df_pivot = df_pivot.sort_values('Growth_Points', ascending=False)
        
        # Reorder columns for clean export
        cols = ['Country_Name', start_col, end_col, 'Growth_Points']
        df_pivot = df_pivot[cols]

        # 9. Export
        df_pivot.to_csv(OUTPUT_FILE, index=False)
        
        print(f"Success. Comparison data exported to {OUTPUT_FILE}")
        print(df_pivot.head(10))

    except Exception as e:
        print(f"Error during processing: {e}")
        sys.exit(1)

if __name__ == "__main__":
    process_race_to_zero()

Analyzing renewable growth (2012-2022) from ../data/raw/nrg_ind_ren_FULL.gz...
Success. Comparison data exported to ../data/output/race_to_zero_growth.csv
Year Country_Name  Value_2012  Value_2022  Growth_Points
11         Sweden      49.403      66.287         16.884
2         Denmark      25.465      41.368         15.903
5         Finland      34.222      47.740         13.518
8     Netherlands       4.659      15.291         10.632
3           Spain      14.239      21.837          7.598
1         Germany      13.549      20.829          7.280
6          France      13.239      20.330          7.091
4      EU Average      16.002      23.003          7.001
0         Belgium       7.086      13.831          6.745
9          Poland      10.955      16.644          5.689
